In [1]:
import django
from django.conf import settings
import pathlib
import os
import dotenv
import pandas

In [2]:
BASE_DIR = pathlib.Path('.')

In [3]:
dotenv.load_dotenv(BASE_DIR / '.env')

True

In [4]:
try:
    settings.configure(**{
        'DEBUG': True,
        'BASE_DIR': BASE_DIR,
        'INSTALLED_APPS': [
            'songs'
        ],
        'DATABASES': {
            'default': {
                'ENGINE': 'django.db.backends.sqlite3',
                'NAME': BASE_DIR / 'db.sqlite3',
            }
        },
        'MEDIA_PATH' : BASE_DIR / 'media'
    })
except:
    pass

In [5]:
os.environ.setdefault('DJANGO_ALLOW_ASYNC_UNSAFE', 'true')

'true'

In [6]:
try:
    django.setup()
except:
    pass

In [7]:
from songs.models import Song, Artist

In [45]:
df = pandas.read_csv(settings.MEDIA_PATH / 'base_artists.csv')

In [46]:
df = df.sort_values('name')

In [43]:
no_duplicates_df = df.drop_duplicates(['name'])

In [44]:
no_duplicates_df.to_csv(settings.MEDIA_PATH / 'base_artists_v2.csv', index=False)

## Completing Spotify metadata

In [16]:
import time
from blindtest.rapidapi.client import Spotify

In [54]:
qs = Song.objects.order_by('artist').filter(
    genre__icontains='disco', spotify_id__isnull=True).values_list('artist', flat=True)
unique_artists = set(qs)
len(unique_artists)

# Song.objects.filter(spotify_id__isnull=True).values_list('genre', flat=True)

3

In [44]:
def spotify_data(genre):
    qs = Song.objects.order_by('artist').filter(genre__icontains=genre, spotify_id__isnull=True).values_list('artist', flat=True)
    unique_artists = set(qs)

    print(f'Searching for {len(unique_artists)}')

    for name in unique_artists:
        instance = Spotify(name)

        try:
            instance.send()
        except:
            time.sleep(5)
            continue

        print(f'Got: {name}')

        try:
            data = instance[0]['data']
        except:
            print(f'Failed to get {name}')
            time.sleep(5)
            continue
        else:
            qs2 = Song.objects.filter(artist=name)
            spotify_id = data['uri'].split(':')[-1]
            
            try:
                avatar = data['visuals']['avatarImage']['sources'][0]['url']
            except:
                avatar = None

            qs2.update(spotify_id=spotify_id, spotify_avatar=avatar)

        time.sleep(5)

In [55]:
spotify_data('disco')

Searching for 3
Got: Los Del Rio
Got: Daft Punk
Got: Village People


## Creating Windows

In [10]:
from django.db.models.functions.window import Rank
from django.db.models import Window, F, Count

In [ ]:
qs = Song.objects.all()

In [185]:
genre_count = Song.objects.values('genre').annotate(count=Count('genre')).order_by('-count')

In [186]:
genre_df = pandas.DataFrame(genre_count)
genre_df.head()

,genre,count
0,Pop rock,32
1,Rhythm and blues,5
2,Rap music,3
3,Electropop,2
4,Techno,1


In [191]:
window = Window(Rank(), order_by='-count')
window_genre = Song.objects.values('genre').annotate(count=Count('genre')).annotate(rank=window).order_by('-count')

In [192]:
df = pandas.DataFrame(window_genre)

In [195]:
df = df.sort_values(by='rank', ascending=True)

In [196]:
df

,genre,count,rank
0,Pop rock,32,1
1,Rhythm and blues,5,2
2,Rap music,3,3
3,Electropop,2,4
4,Techno,1,5
5,Soft rock,1,5
6,Ragga,1,5
7,Pop soul,1,5
8,Indie pop,1,5
9,Gothic rock,1,5
